### Tools in Langchain

Models can request to call tools that perform tasks such as fetching data from a database, searchin the web or running code. 
Tools are a combination of - 
1. A schema, including the name, description and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute

In [1]:
from configs.model_config import model

In [2]:
from langchain.tools import tool

In [3]:
@tool
def get_weather(location:str)->str:
    """
    Get the weather at a specific location
    """
    return f"It's sunny in {location}!"

In [4]:
model_with_tools = model.bind_tools([get_weather])

In [7]:
from langchain.agents import create_agent

weather_agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant!"
)

In [8]:
response = model_with_tools.invoke("What's the weather like in Bangalore?")
print(response)

content='' additional_kwargs={'reasoning_content': 'User asks current weather in Bangalore. Need to call get_weather function.', 'tool_calls': [{'id': 'fc_bd80c40f-c8fe-4111-97bf-dd72667e7f3c', 'function': {'arguments': '{"location":"Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 128, 'total_tokens': 171, 'completion_time': 0.102262043, 'completion_tokens_details': {'reasoning_tokens': 15}, 'prompt_time': 0.007283287, 'prompt_tokens_details': None, 'queue_time': 0.046887833, 'total_time': 0.10954533}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e10890e4b9', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c42d7-af4f-7860-a02d-e2c08fd4af44-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bangalore'}, 'id': 'fc_bd80c40f-c8fe-4111-97bf-dd72667e7f3c', 'type': 'tool_call'}] invalid_tool_calls=[] usage_m

In [9]:
for tool_call in response.tool_calls:
    print(f"Tool : {tool_call['name']}")
    print(f"Args : {tool_call['args']}")

Tool : get_weather
Args : {'location': 'Bangalore'}


In [13]:
agent_response = weather_agent.invoke({"messages":"What's the weather like in Bangalore?"})

In [16]:
print(agent_response["messages"][-1].content)

The current weather in Bangalore is sunny. Enjoy the bright skies!


### Final Method

Binding tools and executing them

This pattern handles the full cycle:

1. Define tools with the @tool decorator (with schema)

2. Bind tools to the LLM via .bind_tools()

3. Call the LLM with user input

4. Check if it wants to call a tool

5. Execute all the tool calls

6. Send the tool results back into the model

7. Produce the final natural language answer

Exact sequence - 
- HumanMessage
- AIMessage (with tool_calls)
- ToolMessage (with tool_call_id + name)
- AIMessage (final answer)

This gives you a clean tool execution loop without using agents — ideal for production control.

### Tool Execution Loop

In [40]:
# Step 1: Model generates tool calls
messages = [
    {
        "role":"user",
        "content":"Can I plan a pool party in Bangalore?"
    }
]

ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

In [41]:
# Step 2: Execute tools and collect the results
for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

In [42]:
# Step 3: Pass results back to the model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

Planning a pool party in Bangalore is definitely doable, but you’ll want to keep a few weather‑related factors in mind to make sure the day stays fun and dry.

### 1. Today’s Weather
- **Current forecast:** It’s raining heavily in Bangalore right now. A pool party today would be uncomfortable (and potentially unsafe) unless you have a fully covered, indoor‑heated pool.

### 2. Seasonal Overview for Bangalore

| Season | Approx. Months | Typical Weather | Pool‑Party Tips |
|--------|----------------|-----------------|-----------------|
| **Winter (Cool & Dry)** | Dec – Feb | Pleasant daytime temps 20‑26 °C, low humidity, clear skies | **Best time** for outdoor pool parties. Evenings can get a bit chilly, so consider a few blankets or a light heater. |
| **Summer (Warm & Sunny)** | Mar – May | Daytime temps 30‑35 °C, occasional thunderstorms | Great for swimming, but bring plenty of shade, sunscreen, and plenty of water. A misting system or fans can keep guests comfortable. |
| **Monsoon

In [43]:
messages

[{'role': 'user', 'content': 'Can I plan a pool party in Bangalore?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks: "Can I plan a pool... in Bangalore?" They likely want advice on planning a pool party, maybe suggestions for venues, permits, weather, etc. Provide guidance. Also could check weather. Could ask if they need weather forecast. Could provide steps: venue selection, permits, catering, entertainment, safety, decorations, timeline, budget. Also note Bangalore weather: generally moderate, but monsoon etc. Could use function get_weather for Bangalore. Let\'s fetch current weather.', 'tool_calls': [{'id': 'fc_ea59997b-c3d8-46f4-bc84-4a2ba01d4050', 'function': {'arguments': '{"location":"Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 130, 'total_tokens': 257, 'completion_time': 0.273327134, 'completion_tokens_details': {'reasoning_tokens': 99}, 'prompt_time'